## Prediction 
following we will predict how many likes and retweets a tweet is expected to have after 30 Min. <br>
1. Load the data 
2. Train the model with the data set 
3. Safe Models one for Retweet prediction one for like prediction

In [1]:
import pandas as pd
#Load File to train the modell 
file_name = './merge_new.json'

df = pd.read_json(file_name)
print('Size before first cleaning: ' + str(len(df)))
#Configure the data so that the modell is accurate 


df = df.rename({'likes 30': 'likes_last'}, axis='columns') #rename column for, makes future work easyer 

df = df[df.counter == 30] #every tweets needs 30 crwal intervalls 

df = df[df.text.str.contains('RT', case=True, regex=True) == False] #dropping the retweets for the training otherwise there are to many outlaiers

df = df[df.name != 'janboehm'] #drop Jan Böhmerman bec. he's too famouse 
print('Size after first cleaning: ' + str(len(df)))


Size before first cleaning: 17458
Size after first cleaning: 12970


In [2]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
X = df[['retweets', 'retweets 4', 'followers']]
Y = df[['retweets 30']]
score = []

X_l = df[['likes', 'likes 4', 'followers']]
Y_l = df[['likes_last']]
score_l = []

for i in range(300):
    
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.25)
    
    X_l_train, X_l_test, y_l_train, y_l_test = train_test_split(X_l, Y_l, test_size = 0.25)




    pf = PolynomialFeatures(degree = 2, include_bias = False)
    pf.fit(X_train)

    X_train_transformed = pf.transform(X_train)
    X_test_transformed = pf.transform(X_test)

    model = LinearRegression()
    model.fit(X_train_transformed, y_train)

    score.append(model.score(X_test_transformed, y_test))
    
    
    pf_l = PolynomialFeatures(degree = 2, include_bias = False)
    pf_l.fit(X_l_train)

    X_l_train_transformed = pf.transform(X_l_train)
    X_l_test_transformed = pf.transform(X_l_test)

    model_l = LinearRegression()
    model_l.fit(X_l_train_transformed, y_l_train)

    score_l.append(model_l.score(X_l_test_transformed, y_l_test))
    
    
print('R^2 Retweets= ' + str(sum(score)/len(score)))
print('R^2 likes= ' + str(sum(score_l)/len(score_l)))

R^2 Retweets= 0.9157629067937368
R^2 likes= 0.9795724822682852


##  Train the model and safe traind model to file

In [3]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.externals import joblib
X = df[['retweets', 'retweets 4', 'followers']]
Y = df[['retweets 30']]
score = []

X_l = df[['likes', 'likes 4', 'followers']]
Y_l = df[['likes_last']]
score_l = []


X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.25)
    
X_l_train, X_l_test, y_l_train, y_l_test = train_test_split(X_l, Y_l, test_size = 0.25)




pf = PolynomialFeatures(degree = 2, include_bias = False)
pf.fit(X_train)

X_train_transformed = pf.transform(X_train)
X_test_transformed = pf.transform(X_test)

model = LinearRegression()
model.fit(X_train_transformed, y_train)

score.append(model.score(X_test_transformed, y_test))


pf_l = PolynomialFeatures(degree = 2, include_bias = False)
pf_l.fit(X_l_train)

X_l_train_transformed = pf.transform(X_l_train)
X_l_test_transformed = pf.transform(X_l_test)

model_l = LinearRegression()
model_l.fit(X_l_train_transformed, y_l_train)

score_l.append(model_l.score(X_l_test_transformed, y_l_test))

joblib.dump(model, 'retweet.model')
joblib.dump(model_l, 'like.model')

print('R^2 likes= ' + str(score_l[0]))
print('R^2 Retweets= ' + str(score[0]))

### IF IT IS NOT POSSIBLE TO DUMP THE MODEL IT IS BEC. SKLEARN REMOVED THE JOBLIB LIBRARY THEN YOU NEED TO IMPORT IT
## FROM JOBLIB DIRECTLY

R^2 likes= 0.9817367643269785
R^2 Retweets= 0.904850301513469


/home/student/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)
